In [65]:
class QuantComp:
    def __init__(self, classical):
        self.classical = classical
        
    def convert(self, cboard):
        ret = []
        for i in range(len(cboard)):
            if cboard[i] == 1:
                hold = str(bin(i).replace("0b",""))
                while(len(hold) < 7):
                    hold = '0' + hold
                ret.append(hold)
        return ret
    
    def setup(self, array):
        tot = len(array)
        n = 7
        s = []
        for i in range(n-1):
            s.append(i)
        if tot > 8:
            rep = 2
        elif tot > 4:
            rep = 3
        elif tot == 4:
            rep = 4
        elif tot == 3:
            rep = 5
        elif tot == 2:
            rep = 6
        else:
            rep = 8
        return rep, n, tot, s
        
    def build_oracle(self, circuit, solutions, n, tot, s):
        for i in range(tot):
            temp = solutions[i]
            for j, yesno in enumerate(reversed(temp)):
                if yesno == '0':
                    circuit.x(j)
            circuit.h(n-1)
            circuit.mct(s, n-1)
            circuit.h(n-1)
            for k, noyes in enumerate(reversed(temp)):
                if noyes == '0':
                    circuit.x(k)
                    
    def amplify(self, circuit, n, s):
        circuit.h(range(n))
        circuit.x(range(n))
        circuit.h(n-1)
        circuit.mct(s, n-1)
        circuit.h(n-1)
        circuit.x(range(n))
        circuit.h(range(n))
        
    def guess(self):
        sol = self.convert(self.classical)
        rep, n, tot, s = self.setup(sol)
        qc = QuantumCircuit(n)
        qc.h(range(n))
        for i in range(rep):
            self.build_oracle(qc, sol, n, tot, s)
            self.amplify(qc, n, s)
        qc.measure_all()
        simulator = Aer.get_backend('qasm_simulator')
        stng = list(execute(qc, backend = simulator, shots = 1).result().get_counts().keys())[0]
        print(int(stng, 2))

In [15]:
from qiskit import *

In [69]:
## Takes in a 1-D array of positions and spits out an index to guess
one = [0,0,1,0,0,0,0,1,0,0,
       0,0,1,0,1,1,1,1,0,0,
       1,0,1,0,0,0,0,0,0,0,
       1,0,0,0,0,0,0,0,0,0,
       1,0,0,0,0,0,0,0,0,0,
       1,0,0,0,0,0,0,0,0,0,
       0,0,0,0,0,0,0,0,0,0,
       0,0,0,1,1,1,1,1,0,0]
two = QuantComp(one)
two.guess()

20
